In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize_scalar

In [ ]:
#Load the data from Elmer/ice model

df = pd.read_csv('elmer-model-output.csv')
df

In [ ]:
#Create Fermat prediction plot for Figure 3a

#Scroll down to variables b0, b1, b2 and m0, m1, m2 
#Refer to values in "elmer-model-output.csv" for correct inputs
#default values are commented out

#This is set up for Greenland. Coordinates need to be changed for Antarctica

x0 = 1500 #meters
y0 = 0

x1 = (-2500 - 767.949) / 2 #meters
y1 = (14330.127 + 15330.127) / 2

x2 = (3767.949 + 5500) / 2 #meters
y2 = (15330.127 + 14330.127) / 2

#Glacier boundaries
x3, y3 = 0, 10000
x4, y4 = -2500, 14330.127
x5, y5 = -767.949, 15330.127
x7, y7 = 3767.949, 15330.127
x8, y8 = 5500, 14330.127
x9, y9 = 3000, 10000

w_0 = 3000 #meters
w_1 = 2000
w_2 = 2000

h_0 = 500 #meters
h_1 = 500 
h_2 = 500 

u_0 = 175 #meters per year

Q_0 = u_0 * h_0 * w_0 #cubic meters per year
Q_1 = 0.5 * Q_0
Q_2 = 0.5 * Q_0

#u_0 = Q_0 / (h_0 * w_0)
u_1 = Q_1 / (h_1 * w_1) #meters per year
u_2 = Q_2 / (h_2 * w_2)

#Basal friction parameter: Bifurcation point is very sensitive to this
#default_b = 1e-5
b0 = 
b1 = 
b2 = 

#Shear margin strength parameter:
#default_m = 5e-3
m0 =  
m1 =  
m2 = 

A = (5.15e-17) #[Pa^-3][y^-1] warm ice

#Calculate dissipations
D0 = (b0 * (Q_0)**2 / (w_0 * h_0**2) + 
             m0 * ((Q_0)**(4/3) * h_0**(2/3) / (A**(1/3) * w_0**(8/3))))
D1 = (b1 * (Q_1)**2 / (w_1 * h_1**2) + 
             m1 * ((Q_1)**(4/3) * h_1**(2/3) / (A**(1/3) * w_1**(8/3))))
D2 = (b2 * (Q_2)**2 / (w_2 * h_2**2) + 
             m2 * ((Q_2)**(4/3) * h_2**(2/3) / (A**(1/3) * w_2**(8/3))))


#Find the minimum dissipation point
min_r = float('inf')
min_x = 0.

def dissipation(x):
    yb = 10000.
    r = D0 * np.sqrt((x - x0)**2 + (yb - y0)**2) + D1 * np.sqrt((x - x1)**2 + (yb - y1)**2) + D2 * np.sqrt((x - x2)**2 + (yb - y2)**2)
    return r

out = minimize_scalar(dissipation, bounds = (0, 3000), method = 'bounded')
min_x = out.x

yb = 10000.
#Calculate the bifurcation angle
m1 = (y1-yb)/(x1-min_x)
m2 = (y2-yb)/(x2-min_x)
theta = np.degrees(np.arctan(np.abs((m2 - m1)/(1+m1*m2))))

#Make the plot
fig, ax = plt.subplots(figsize = (6.55,8.4))

#Fermat Triangle
lw = 1
c = "#0072B2"
x01 = np.linspace(x1, x0, 100)
y01 = (y0-y1) * (x01 - x1) / (x0 - x1) + y1
ax.plot(x01, y01, color = c, linestyle = 'dashed', linewidth = lw)

x12 = np.linspace(x1, x2, 100)
y12 = (y1-y2) * (x12 - x2) / (x1 - x2) + y2
ax.plot(x12, y12, color = c, linestyle = 'dashed', linewidth = lw)

x02 = np.linspace(x0, x2, 100)
y02 = (y0-y2) * (x02 - x2) / (x0 - x2) + y2
ax.plot(x02, y02, color = c, linestyle = 'dashed', linewidth = lw)

#Dissipation Lines
c = "#E69F00"
lw = 2.5
x0b = np.linspace(x0, min_x, 100)
if (x0 - min_x) == 0:
    ax.plot([x0, x0], [y0, yb], color = c, linewidth = lw)
else:
    y0b = (y0-yb) * (x0b - min_x) / (x0 - min_x) + yb
    ax.plot(x0b, y0b, color = c, linewidth = lw)

x1b = np.linspace(x1, min_x, 100)
y1b = (y1-yb) * (x1b - min_x) / (x1 - min_x) + yb
ax.plot(x1b, y1b, color = c, linewidth = lw)

x2b = np.linspace(min_x, x2, 100)
y2b = (y2-yb) * (x2b - min_x) / (x2 - min_x) + yb
ax.plot(x2b, y2b, color = c, linewidth = lw)

#Background dissipation lines
c = "#E69F00"
lw = 2.5
a = 0.35
x0b = np.linspace(x0, x9//2, 100)
if (x0 - x9//2) == 0:
    ax.plot([x0, x0], [y0, yb], color = c, linewidth = lw, alpha = a)
else:
    y0b = (y0-yb) * (x0b - x9//2) / (x0 - x9//2) + yb
    ax.plot(x0b, y0b, color = c, linewidth = lw, alpha = a)

x1b = np.linspace(x1, x9//2, 100)
y1b = (y1-yb) * (x1b - x9//2) / (x1 - x9//2) + yb
ax.plot(x1b, y1b, color = c, linewidth = lw, alpha = a)

x2b = np.linspace(x9//2, x2, 100)
y2b = (y2-yb) * (x2b - x9//2) / (x2 - x9//2) + yb
ax.plot(x2b, y2b, color = c, linewidth = lw, alpha = a)

#Points
ax.plot(x0, y0, 'ko')
ax.plot(x1, y1, 'ko')
ax.plot(x2, y2, 'ko')
ax.plot(min_x, yb, 'kv', markersize=10, label='Stable\nBifurcation\nPoint')
ax.plot(x0, yb, 'kv', alpha = a, markersize=10, label='Stable\nBifurcation\nPoint')

lw = 0.75
ls = 'solid'
plt.vlines([0, x9], ymin = 0, ymax = y3, color = 'k', linewidth = lw, linestyle = ls)
plt.hlines(0, xmin = 0, xmax = x9, color = 'k', linewidth = lw, linestyle = ls)

#bifurcation start
plt.hlines(y3, xmin = 0, xmax = x9, linestyle = 'dashed', color = 'k', linewidth = lw)

def find_slope(x1, y1, x2, y2):
    m = (y2-y1)/(x2 - x1)
    return m

xtmp = np.arange(x4, x3)
plt.plot(xtmp, find_slope(x3, y3, x4, y4) * (xtmp - x4) + y4, color = 'k', linewidth = lw, linestyle = ls) 
xtmp = np.arange(x4, x5)
plt.plot(xtmp, find_slope(x4, y4, x5, y5) * (xtmp - x4) + y4, color = 'k', linewidth = lw, linestyle = ls) 
xtmp = np.arange(x5, 1500)
plt.plot(xtmp, find_slope(x3, y3, x4, y4) * (xtmp - x5) + y5, color = 'k', linewidth = lw, linestyle = ls)
xtmp = np.arange(1500, x7)
plt.plot(xtmp, find_slope(x9, y9, x8, y8) * (xtmp - x7) + y7, color = 'k', linewidth = lw, linestyle = ls)
xtmp = np.arange(x7, x8)
plt.plot(xtmp, find_slope(x7, y7, x8, y8) * (xtmp - x7) + y7, color = 'k', linewidth = lw, linestyle = ls) 
xtmp = np.arange(x9, x8)
plt.plot(xtmp, find_slope(x9, y9, x8, y8) * (xtmp - x9) + y9, color = 'k', linewidth = lw, linestyle = ls)

#Annotations
ax.annotate("$\\mathbf{P_0}$", xy = (x0-300,y0-700), fontsize = 14)
ax.annotate("$\\mathbf{P_1}$", xy = (x1-800,y1+100), fontsize = 14)
ax.annotate("$\\mathbf{P_2}$", xy = (x2+100,y2+100), fontsize = 14)
ax.annotate("$\\mathbf{P_B}$", xy = (min_x+50,yb-500), fontsize = 14)

ax.annotate("$D_0$", xy = ((x0+min_x)/2+100,(y0+yb)/2+0.02), fontsize = 12)
ax.annotate("$D_1$", xy = ((x1+min_x)/2-900,(y1+yb)/2), fontsize = 12)
ax.annotate("$D_2$", xy = ((x2+min_x)/2-700,(y2+yb)/2+0.05), fontsize = 12)

#Axes
ax.axis('equal')
ax.set_axis_off()
plt.xticks(ticks = [-3000, -1500, 0, 1500, 3000, 4500, 6000], labels = [-4500, -3000, -1500, 0, 1500, 3000, 4500])
plt.yticks([0, 2500, 5000, 7500, 10000, 12500, 15000])
plt.xlim([-100, 3100])
plt.ylim([-1000, 16000])

#Axes labels
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
sns.despine()
#plt.savefig('elmer-blank.png', dpi = 300, bbox_inches = 'tight', transparent = True)
plt.show()